<a href="https://colab.research.google.com/github/yashrajkakkad/ml-spotify/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/My Drive/ml-spotify/data_merged.csv')
df.head()

In [ ]:
df.columns

In [ ]:
mean_popularity = 42
df["popularity"] = [ 1 if i >= mean_popularity else 0 for i in df.popularity ]
df.popularity.value_counts()  

In [ ]:
y = df["popularity"].values
X = df.drop(["popularity", "mode", "loudness", "loudness_ar", "loudness_yr"],axis=1)

In [ ]:
X.shape, y.shape

In [ ]:
%%time

from sklearn.feature_selection import SelectPercentile, chi2
X = SelectPercentile(chi2, percentile=10).fit_transform(X, y)

input_scaler = StandardScaler()
X_normalized = input_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.20, random_state=42, stratify=y
)

In [ ]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from joblib import dump


svm = SVC(kernel='linear',
          C=1000,
          random_state=42)
svm.fit(X_train[:10000], y_train[:10000])
dump(svm, "mlspotify_SVM")

print("Train accuracy of svm algo:", svm.score(X_train[:10000], y_train[:10000]))
print("Test accuracy of svm algo:", svm.score(X_test, y_test))

# cv_scores = cross_val_score(svm, X_train[:20000], y_train[:20000])
# print("Cross Validation scores - ", cv_scores)

# y_train_pred = cross_val_predict(svm, X_train[:20000], y_train[:20000])

# confusion_matrix = confusion_matrix(y_train[:20000], y_train_pred)
# print("Confusion matrix\n", confusion_matrix)

# p_score = precision_score(y_train[:20000], y_train_pred)
# rc_score = recall_score(y_train[:20000], y_train_pred)
# f1_score = f1_score(y_train[:20000], y_train_pred)
# p_score, rc_score, f1_score
# print("Recall score = ", rc_score)
# print("Precision score = ", p_score)
# print("F1 score = ", f1_score)

y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))

In [ ]:
from joblib import load
clf = load("mlspotify_SVM")
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))